In [55]:
from ase.visualize import view

from ase import units
from gpaw import GPAW
from ase.calculators.emt import EMT
from ase.lattice.cubic import FaceCenteredCubic
from ase.md.verlet import VelocityVerlet
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

##### building structure

In [56]:
cu_strc = FaceCenteredCubic(
    directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]], symbol="Cu", size=(3, 3, 3), pbc=True
)

# view(cu_strc)

In [57]:
cu_strc

Lattice(symbols='Cu108', pbc=True, cell=[10.83, 10.83, 10.83])

In [58]:
cu_strc.calc = EMT()

In [59]:
cu_strc

Lattice(symbols='Cu108', pbc=True, cell=[10.83, 10.83, 10.83], calculator=EMT(...))

In [60]:
MaxwellBoltzmannDistribution(atoms=cu_strc, temperature_K=300)

In [61]:
cu_strc

Lattice(symbols='Cu108', pbc=True, cell=[10.83, 10.83, 10.83], momenta=..., calculator=EMT(...))

In [62]:
dyn = VelocityVerlet(cu_strc, 5 * units.fs, trajectory='cu_strc.traj')

In [63]:
def printenergy(a):
    """Function to print the potential, kinetic and total energy"""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print(
        f'Energy per atom: Epot ={epot:6.3f}eV  Ekin = {ekin:.3f}eV '
        f'(T={ekin / (1.5 * units.kB):3.0f}K) Etot = {epot + ekin:.3f}eV'
    )

In [64]:
printenergy(cu_strc)
for i in range(30):
    dyn.run(10)
    printenergy(cu_strc)

Energy per atom: Epot =-0.006eV  Ekin = 0.039eV (T=303K) Etot = 0.033eV


Energy per atom: Epot = 0.025eV  Ekin = 0.009eV (T= 68K) Etot = 0.034eV
Energy per atom: Epot = 0.014eV  Ekin = 0.020eV (T=151K) Etot = 0.034eV
Energy per atom: Epot = 0.013eV  Ekin = 0.020eV (T=157K) Etot = 0.034eV
Energy per atom: Epot = 0.011eV  Ekin = 0.023eV (T=177K) Etot = 0.034eV
Energy per atom: Epot = 0.016eV  Ekin = 0.018eV (T=137K) Etot = 0.034eV
Energy per atom: Epot = 0.015eV  Ekin = 0.019eV (T=145K) Etot = 0.034eV
Energy per atom: Epot = 0.011eV  Ekin = 0.023eV (T=178K) Etot = 0.034eV
Energy per atom: Epot = 0.017eV  Ekin = 0.016eV (T=126K) Etot = 0.034eV
Energy per atom: Epot = 0.010eV  Ekin = 0.023eV (T=178K) Etot = 0.034eV
Energy per atom: Epot = 0.015eV  Ekin = 0.019eV (T=144K) Etot = 0.034eV
Energy per atom: Epot = 0.014eV  Ekin = 0.020eV (T=151K) Etot = 0.034eV
Energy per atom: Epot = 0.017eV  Ekin = 0.017eV (T=130K) Etot = 0.034eV
Energy per atom: Epot = 0.007eV  Ekin = 0.027eV (T=208K) Etot = 0.034eV
Energy per atom: Epot = 0.021eV  Ekin = 0.013eV (T=101K) Etot = 

In [72]:
from ase.io.trajectory import read_traj, Trajectory

In [ ]:
traj = Trajectory('cu_strc.traj')

view(traj)

<Popen: returncode: None args: ['/home/cer/.ase_venv/bin/python', '-m', 'ase...>

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/cer/.ase_venv/lib/python3.12/site-packages/ase/gui/pipe.py", line 34, in <module>
    main()
  File "/home/cer/.ase_venv/lib/python3.12/site-packages/ase/gui/pipe.py", line 30, in main
    plt.show()
  File "/home/cer/.ase_venv/lib/python3.12/site-packages/matplotlib/pyplot.py", line 613, in show
    return _get_backend_mod().show(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cer/.ase_venv/lib/python3.12/site-packages/matplotlib_inline/backend_inline.py", line 90, in show
    display(
  File "/home/cer/.ase_venv/lib/python3.12/site-packages/IPython/core/display_functions.py", line 245, in display
    print(*objs)
ValueError: I/O operation on closed file.


In [70]:
filee  = read_traj('cu_strc.traj', index=['@:'])

view(filee)

AttributeError: 'list' object has no attribute 'indices'